In [12]:
from langchain.chat_models import ChatOpenAI 
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader

# loader = TextLoader("./files/chapter_one.txt")
# loader = PyPDFLoader("./files/chapter_one.pdf")
loader = UnstructuredFileLoader("./files/chapter_one.pdf")

# 문서가 너무 크면 나누는 작업을 해야 함.
len(loader.load()) 



1